# Imports

In [1]:
import json
import requests

from datetime import datetime, timedelta, timezone, time, date
from itertools import product
from functools import reduce

import pandas as pd
import numpy as np

In [2]:
# import functions from eclipses notebook
%run desktop/opentransit-metrics/mykelu/eclipses.py

# Get Stops

In [3]:
def get_stops(dates, routes, directions = [], new_stops = [], times = ("00:00", "23:59")):
    """
    get_stops
    
    Description:
        Returns every instance of a bus stopping at a given set of stops, on a given set of routes, during a given time period.

    Parameters:
        dates: an array of dates, formatted as strings in the form YYYY-MM-DD
        routes: an array of routes, each represented as a string
        directions: an array of strings representing the directions to filter
        stops: an array of strings representing the stops to filter
        times: a tuple with the start and end times (in UTC -8:00) as strings in the form HH:MM 

    Returns:
        stops: a DataFrame, filtered by the given directions and stops, with the following columns:
            VID: the vehicle ID
            Time: a datetime object representing the date/time of the stop
            Route: the route on which the stop occurred
            Stop: the stop at which the stop occurred
            Dir: the direction in which the stop occurred
    """
    bus_stops = pd.DataFrame(columns = ["VID", "DATE", "TIME", "SID", "DID", "ROUTE"])
    
    for route in routes:
        stop_ids = [stop['id']
            for stop
            in requests.get(f"http://restbus.info/api/agencies/sf-muni/routes/{route}").json()['stops']][2:4]

        for stop_id in stop_ids:
            # check if stops to filter were provided, or if the stop_id is in the list of filtered stops
            if (stop_id in new_stops) ^ (len(new_stops) == 0):
                for date in dates:
                    print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: starting processing on stop {stop_id} on route {route} on {date}.")
                    start_time = int(datetime.strptime(f"{date} {timespan[0]} -0800", "%Y-%m-%d %H:%M %z").timestamp())*1000
                    end_time   = int(datetime.strptime(f"{date} {timespan[1]} -0800", "%Y-%m-%d %H:%M %z").timestamp())*1000

                    data = query_graphql(start_time, end_time, route)
                    print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: performed query.")
                          
                    if data is None:  # API might refuse to cooperate
                        print("API probably timed out")
                        continue
                    elif len(data) == 0:  # some days somehow have no data
                        print(f"no data for {month}/{day}")
                        continue
                    else:
                        stops = produce_stops(data, route)
                        print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: produced stops.")
                              
                        buses = produce_buses(data)
                        print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: produced buses.")

                        stop = stops[stops['SID'] == stop_id].squeeze()
                        buses = buses[buses['DID'] == stop['DID']]

                        eclipses = find_eclipses(buses, stop)
                        print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: found eclipses.")
                              
                        nadirs = find_nadirs(eclipses)
                        print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: found nadirs.")
                            
                        nadirs["TIME"] = nadirs["TIME"].apply(lambda x: datetime.fromtimestamp(x//1000, timezone(timedelta(hours = -8))))
                        nadirs['DATE'] = nadirs['TIME'].apply(lambda x: x.date())
                        nadirs['TIME'] = nadirs['TIME'].apply(lambda x: x.time())
                        nadirs["SID"] = stop_id
                        nadirs["DID"] = stop["DID"]
                        nadirs["ROUTE"] = route
                        old_length = len(bus_stops)
                        bus_stops = bus_stops.append(nadirs, sort = True)
                        print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: finished processing.")

    # filter for directions
    if len(directions) > 0:
        bus_stops = bus_stops.loc[bus_stops['DID'].apply(lambda x: x in directions)]
                              
    # prepare timestamp data
    bus_stops['timestamp'] = bus_stops[['DATE', 'TIME']].apply(lambda x: datetime.strptime(f"{x['DATE'].isoformat()} {x['TIME'].isoformat()} -0800", 
                                                                                       "%Y-%m-%d %H:%M:%S %z"), axis = 'columns')
    
    return bus_stops

In [4]:
# get some stops
route = ["12", "14"]

timespan = ("08:00",
            "11:00")

dates = [
    "2018-11-12",
    "2018-11-13",
    "2018-11-14",
    "2018-11-15",
    "2018-11-16",
]

new_stops = get_stops(dates, route, times = timespan)

Mon Feb 04 03:22:19 PM: starting processing on stop 5851 on route 12 on 2018-11-12.
Mon Feb 04 03:22:31 PM: performed query.
Mon Feb 04 03:22:34 PM: produced stops.
Mon Feb 04 03:22:34 PM: produced buses.
Mon Feb 04 03:22:34 PM: found eclipses.
Mon Feb 04 03:22:34 PM: found nadirs.
Mon Feb 04 03:22:35 PM: finished processing.
Mon Feb 04 03:22:35 PM: starting processing on stop 5851 on route 12 on 2018-11-13.
Mon Feb 04 03:22:43 PM: performed query.
Mon Feb 04 03:22:44 PM: produced stops.
Mon Feb 04 03:22:44 PM: produced buses.
Mon Feb 04 03:22:44 PM: found eclipses.
Mon Feb 04 03:22:44 PM: found nadirs.
Mon Feb 04 03:22:44 PM: finished processing.
Mon Feb 04 03:22:44 PM: starting processing on stop 5851 on route 12 on 2018-11-14.
Mon Feb 04 03:22:54 PM: performed query.
Mon Feb 04 03:22:54 PM: produced stops.
Mon Feb 04 03:22:54 PM: produced buses.
Mon Feb 04 03:22:55 PM: found eclipses.
Mon Feb 04 03:22:55 PM: found nadirs.
Mon Feb 04 03:22:55 PM: finished processing.
Mon Feb 04 03:22

In [5]:
# TODO: parse direction as inbound/outbound? (remove route indicator)
# test directions (find unique directions) - possibly parse directions
new_stops.head()

,DATE,DID,ROUTE,SID,TIME,VID,timestamp
1,2018-11-12,12___O_F00,12,5851,08:00:13,8776,2018-11-12 08:00:13-08:00
5270,2018-11-12,12___O_F00,12,5851,10:28:06,8776,2018-11-12 10:28:06-08:00
79,2018-11-12,12___O_F00,12,5851,08:02:13,8705,2018-11-12 08:02:13-08:00
4835,2018-11-12,12___O_F00,12,5851,10:16:05,8705,2018-11-12 10:16:05-08:00
979,2018-11-12,12___O_F00,12,5851,08:28:00,8912,2018-11-12 08:28:00-08:00


# Prepare Timestamp Data

In [6]:
# # TODO: when to use date/time columns vs timestamps?
# new_stops['timestamp'] = new_stops[['DATE', 'TIME']].apply(lambda x: datetime.strptime(f"{x['DATE'].isoformat()} {x['TIME'].isoformat()} -0800", 
#                                                                                        "%Y-%m-%d %H:%M:%S %z"),  
#                                                            axis = 'columns')

In [7]:
new_stops['timestamp'].head()

1      2018-11-12 08:00:13-08:00
5270   2018-11-12 10:28:06-08:00
79     2018-11-12 08:02:13-08:00
4835   2018-11-12 10:16:05-08:00
979    2018-11-12 08:28:00-08:00
Name: timestamp, dtype: datetime64[ns, UTC-08:00]

# Compute Average Waiting Time

In [8]:
# initial, slower implementation of get_wait_times

# find the smallest nonnegative waiting time
def absmin(series):
    return series[series >= 0].min()

# # input: df with entries from one day
# # possible optimzation: sort df by timestamp, then pick first timestamp > minute for each minute (need to time to make sure but should be faster)
def minimum_waiting_times(df, start_time, end_time, group):
    minute_range = [start_time + timedelta(minutes = i) for i in range((end_time - start_time).seconds//60)]
    wait_times = pd.DataFrame(columns = [])
    
    for minute in minute_range:
        df['WAIT'] = df['timestamp'].apply(lambda x: (x - minute).total_seconds())
        pivot = df[group + ['WAIT']].pivot_table(values = ['WAIT'], index = group, aggfunc = absmin)
        pivot['TIME'] = minute
        pivot = pivot.reset_index()
        wait_times = wait_times.append(pivot, sort = True)
        
    return wait_times

def all_wait_times(df, timespan, group, aggfuncs):
    dates = df['DATE'].unique()
    avg_over_pd = pd.DataFrame(columns = group + ['DATE', 'TIME', 'WAIT'])
    
    for date in new_stops['DATE'].unique():
        print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: start processing {date}.")
        start_time = datetime.strptime(f"{date.isoformat()} {timespan[0]} -0800", "%Y-%m-%d %H:%M %z")
        end_time   = datetime.strptime(f"{date.isoformat()} {timespan[1]} -0800", "%Y-%m-%d %H:%M %z")
        daily_wait = minimum_waiting_times(new_stops[new_stops['DATE'] == date], start_time, end_time, group)
        print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: found stops for {date}.")      
        #daily_wait = daily_wait.pivot_table(values = ['WAIT'], index = group).reset_index()
        daily_wait['DATE'] = date
        daily_wait['TIME'] = daily_wait['TIME'].apply(lambda x: x.time())
        avg_over_pd = avg_over_pd.append(daily_wait, sort = True)
    
    return avg_over_pd#.pivot_table(values = ['WAIT'], index = group, aggfunc = aggfuncs)

In [9]:
all_wait_times(new_stops, timespan, ['SID'], {'WAIT': [np.mean]}).head()

Mon Feb 04 03:25:49 PM: start processing 2018-11-12.


C:\Users\jose\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Mon Feb 04 03:25:58 PM: found stops for 2018-11-12.
Mon Feb 04 03:25:58 PM: start processing 2018-11-13.
Mon Feb 04 03:26:10 PM: found stops for 2018-11-13.
Mon Feb 04 03:26:10 PM: start processing 2018-11-14.
Mon Feb 04 03:26:21 PM: found stops for 2018-11-14.
Mon Feb 04 03:26:21 PM: start processing 2018-11-15.
Mon Feb 04 03:26:33 PM: found stops for 2018-11-15.
Mon Feb 04 03:26:33 PM: start processing 2018-11-16.
Mon Feb 04 03:26:43 PM: found stops for 2018-11-16.


,DATE,SID,TIME,WAIT
0,2018-11-12,5528,08:00:00,223.0
1,2018-11-12,5579,08:00:00,103.0
2,2018-11-12,5844,08:00:00,178.0
3,2018-11-12,5851,08:00:00,13.0
0,2018-11-12,5528,08:01:00,163.0


In [10]:
# a faster implementation of get_wait_times
def get_wait_times(df, dates, timespan, group):
    """
    get_wait_times
    
    Description:
        Takes a DataFrame containing stops for a given route/timespan and returns the corresponding waiting times in that timespan.
        
    Parameters:
        df: a DataFrame containing stop times for a given route/timespan/date interval.
        dates: the range of dates over which to retrieve wait times.
        timespan: the timespan to compute wait times for.
        group: the columns to group over. Needed for sorting.
    
    Returns:
        wait_times: a DataFrame containing the waiting times over the given parameters.
    """
    print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: starting!")
    
    # sort the DataFrame by time first
    df = df.sort_values(['timestamp']).reset_index()
    wait_times = pd.DataFrame(columns = [])      
    filters = product(*[new_stops[s].unique() for s in group])
    filters = [{group[i]:filter[i] for i in range(len(group))} for filter in filters]
          
    # include day range, take from dates       
    for filter in filters:
        filtered_waits = pd.DataFrame(columns = [])
        filtered_stops = df.loc[reduce((lambda x, y: x & y), [df.apply(lambda x: x[key] == filter[key], axis = 1) for key in filter.keys()]), :]
        start_time = datetime.strptime(f"{filter['DATE'].isoformat()} {timespan[0]} -0800", "%Y-%m-%d %H:%M %z")
        end_time   = datetime.strptime(f"{filter['DATE'].isoformat()} {timespan[1]} -0800", "%Y-%m-%d %H:%M %z")
        minute_range = [start_time + timedelta(minutes = i) for i in range((end_time - start_time).seconds//60)]
        current_index = 0

        for minute in minute_range:
            while current_index < len(filtered_stops):
                if filtered_stops.iloc[current_index]['timestamp'] >= minute:
                    break
                else:
                    current_index += 1
                                       
            # catches the case where current_stops = len(filtered_stops)
            try:
                filtered_waits = filtered_waits.append(filtered_stops.iloc[current_index])
            except:
                filtered_waits = filtered_waits.append(filtered_stops.iloc[-1])

        filtered_waits.index = range(len(filtered_waits))
        filtered_waits['MINUTE'] = pd.Series(minute_range)
        wait_times = wait_times.append(filtered_waits)

    wait_times['WAIT'] = wait_times.apply(lambda x: (x['timestamp'] - x['MINUTE']).total_seconds(), axis = 'columns')
    wait_times.index = range(len(wait_times))
    wait_times['MINUTE'] = wait_times['MINUTE'].apply(lambda x: x.time())
    print(f"{datetime.now().strftime('%a %b %d %I:%M:%S %p')}: finishing!")
    return wait_times[['MINUTE', 'WAIT'] + group]

In [11]:
waits = get_wait_times(new_stops, dates, timespan, ['DATE', 'SID'])

Mon Feb 04 03:26:43 PM: starting!
Mon Feb 04 03:27:22 PM: finishing!


In [12]:
# TODO:
# find route combinations to test (w/overlapping stops/directions)
# write other metrics

In [13]:
waits.head()

,MINUTE,WAIT,DATE,SID
0,08:00:00,13.0,2018-11-12,5851
1,08:01:00,73.0,2018-11-12,5851
2,08:02:00,13.0,2018-11-12,5851
3,08:03:00,1500.0,2018-11-12,5851
4,08:04:00,1440.0,2018-11-12,5851


In [14]:
# what should be done to instances of negative wait times?
# need to collect data until a stop has been found for all times in timespan
# don't know how to get this to work w/current graphql query
# => need to rewrite get_stops so that all times have a nonnegative wait?
# ez solution: 
waits[waits.apply(lambda x: x['WAIT'] < 0, axis = 'columns')]

,MINUTE,WAIT,DATE,SID
179,10:59:00,-8.0,2018-11-12,5851
1605,10:45:00,-6.0,2018-11-14,5851
1606,10:46:00,-66.0,2018-11-14,5851
1607,10:47:00,-126.0,2018-11-14,5851
1608,10:48:00,-186.0,2018-11-14,5851
1609,10:49:00,-246.0,2018-11-14,5851
1610,10:50:00,-306.0,2018-11-14,5851
1611,10:51:00,-366.0,2018-11-14,5851
1612,10:52:00,-426.0,2018-11-14,5851
1613,10:53:00,-486.0,2018-11-14,5851


# Get Summary Statistics

In [15]:
# summary stats over all waits
waits.describe()

,WAIT
count,3600.000000
mean,444.547500
std,393.707321
min,-846.000000
25%,165.000000
50%,358.000000
75%,641.000000
max,2764.000000


In [16]:
def quantiles(series):
    return [np.percentile(series, i) for i in [5, 25, 50, 75, 95]]

def get_summary_statistics(df, group):
    waits = df.pivot_table(values = ['WAIT'], index = group, aggfunc = {'WAIT': [np.mean, np.std, quantiles]}).reset_index()
    waits.columns = ['_'.join(col) if col[0] == 'WAIT' else ''.join(col) for col in waits.columns.values]
    waits[[f"{i}th percentile" for i in [5, 25, 50, 75, 95]]] = waits['WAIT_quantiles'].apply(lambda x: pd.Series(x))
    waits = waits.drop('WAIT_quantiles', axis = 1)
    return waits

In [17]:
get_summary_statistics(waits, ['DATE', 'SID'])

,DATE,SID,WAIT_mean,WAIT_std,5th percentile,25th percentile,50th percentile,75th percentile,95th percentile
0,2018-11-12,5528,301.100000,202.153631,33.95,150.50,275.5,431.75,662.55
1,2018-11-12,5579,306.194444,199.570799,42.70,152.50,284.5,453.25,648.70
2,2018-11-12,5844,437.983333,270.492959,51.00,193.25,422.5,659.25,874.85
3,2018-11-12,5851,483.405556,330.778204,15.85,209.50,461.5,720.50,1020.80
4,2018-11-13,5528,371.305556,276.942587,39.00,150.00,305.5,557.00,929.05
5,2018-11-13,5579,367.172222,276.024892,38.95,150.00,299.0,532.00,929.05
6,2018-11-13,5844,634.111111,448.962428,58.95,294.75,538.5,887.50,1545.50
7,2018-11-13,5851,630.300000,445.270226,58.00,294.75,538.5,873.00,1497.50
8,2018-11-14,5528,389.288889,344.584820,35.95,149.25,282.5,551.75,1116.10
9,2018-11-14,5579,372.461111,312.705412,33.95,149.25,278.5,521.00,982.45
